In [6]:
import duckdb
from google.cloud import bigquery

con = duckdb.connect("nps.db")

bqclient = bigquery.Client(project="nps-public-data")

dataset_ref = f"nps-public-data.nps_public_data"

In [7]:
tables = bqclient.list_tables(dataset_ref)

for table in tables:
    print(table.table_id)

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [12]:
schema = "nps_public_data"

con.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")

for table in tables:

    table_str = f"{table.project}.{table.dataset_id}.{table.table_id}"

    table = bigquery.TableReference.from_string(table_str)

    rows = bqclient.list_rows(table)

    # this is a terrible pun
    arr_rows = rows.to_arrow(create_bqstorage_client=True)

    sql_ref = f"{schema}.{table.table_id}"

    con.sql(
        f"""
        DROP TABLE IF EXISTS {sql_ref};
        CREATE TABLE {sql_ref} AS SELECT * FROM arr_rows;
        """
    )

    print(table_str)

con.sql("SHOW ALL tables")

RefreshError: ('invalid_grant: Token has been expired or revoked.', {'error': 'invalid_grant', 'error_description': 'Token has been expired or revoked.'})

In [4]:
dbname = "nps"
con.sql(
    f"EXPORT DATABASE '../data/{dbname}' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000);"
)